# Lidar Reconstruction using Open3D
Library used: http://open3d.org/

## 1. Setup
Open3D installation instructions: http://open3d.org/docs/getting_started.html

In [ ]:
from py3d import *
import numpy as np
import pandas as pd
import copy
import glob

### Importing Data
Imports all .csv data files in `path`

In [2]:
path = "reconstruction-data/"
all_files = sorted(glob.glob(path + "/*.csv"))

## 2. Preprocessing Point Cloud Data 
### Cleaning Data
Remove "reflection" noise near the lidar machine  
Add cleaned data to list as `py3d.PointCloud` objects

In [3]:
cleaned_point_clouds = []
point_cloud_arr = []

for csv_file in all_files:
    df = pd.read_csv(csv_file, 
                 names = ['x', 'y', 'z', 
                          'intensity', 'ring', 'rotation', 'revolution'])
    df['distance'] = np.sqrt(df['x']**2 + df['y']**2 + df['z']**2)
    df = df[df.distance > 2.5]
    point_cloud_matrix = df[['x', 'y', 'z']].as_matrix()
    
    cleaned_point_clouds.append(point_cloud_matrix)
    
    pcd = PointCloud()
    pcd.points = Vector3dVector(point_cloud_matrix)
    
    point_cloud_arr.append(voxel_down_sample(pcd, voxel_size = 0.20)) 

## 3. Registration and Reconstruction
### Defining Utility Functions

In [4]:
def draw_registration_result(source, target, transformation):
    """
    Draws the source point cloud with transformation applied to it (orange)
    and the target point cloud (blue).
    
    Args:
    source: py3d.PointCloud object
    target: py3d.PointCloud object
    transformation: 4x4 affine transformation matrix to be applied to source
    """
    
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    draw_geometries([source_temp, target_temp])

In [5]:
def eval_init_align(source, target, threshold=1,
                    trans_init=np.asarray([[1.0, 0.0, 0.0, 0.0],
                                           [0.0, 1.0, 0.0, 0.0],
                                           [0.0, 0.0, 1.0, 0.0],
                                           [0.0, 0.0, 0.0, 1.0]])):
    """
    Prints evaluation of an initial alignment by the identity transformation matrix.
    
    Args:
    source: py3d.PointCloud object
    target: py3d.PointCloud object
    threshold: distance threshold
    trans_init: initial transformation matrix
    """
    print("Initial Alignment")
    evaluation = evaluate_registration(source, target, 
                                       threshold, trans_init)
    print(evaluation)

In [6]:
def icp_p2l(source, target, radius=.5, max_nn=100, threshold = 1.0,
            trans_init=np.asarray([[1.0, 0.0, 0.0, 0.0],
                                   [0.0, 1.0, 0.0, 0.0],
                                   [0.0, 0.0, 1.0, 0.0],
                                   [0.0, 0.0, 0.0, 1.0]]), max_iter=200):
    """
    Applies point-to-plane iterative closest point (ICP) algorithm
    
    Args:
    source: py3d.PointCloud object
    target: py3d.PointCloud object
    radius: KD Tree search radius
    max_nn: KD Tree maximum nearest neighbors
    threshold: distance threshold
    trans_init: initial transformation matrix
    max_iter: ICP convergence maximum iterations
    
    Returns:
    4x4 affine transformation matrix from ICP
    """
#     print("Compute the normal")
    estimate_normals(target, search_param = KDTreeSearchParamHybrid(
        radius, max_nn))
#     print("Apply point-to-plane ICP")
    reg_p2l = registration_icp(source, target, threshold, trans_init,
                               TransformationEstimationPointToPlane(),
                               ICPConvergenceCriteria(max_iter))
#     print(reg_p2l)
#     print("Transformation is:")
#     print(reg_p2l.transformation)
#     print("")
    return reg_p2l.transformation

### Initial Alignment
Show baseline `fitness` and `inlier_rmse` for initial alignment between 
adjacent point clouds assuming that there is no movement

`fitness` measures the overlapping area (higher is better)  
`inlier_rmse` measures the RMSE of all inlier correspondences (lower is better)

In [ ]:
for i in range(len(point_cloud_arr)-1):
    print("Initial alignment between point cloud " + str(i) + " with point cloud " + str(i+1))
    eval_init_align(point_cloud_arr[i], point_cloud_arr[i+1])

### Calculate Adjacent Transformations using Point-to-Plane ICP
Create array of transformations that align every adjacent point cloud

In [7]:
transformation_arr = []

for i in range(len(point_cloud_arr)-1):
#     print("Aligning point cloud " + str(i) + " with point cloud " + str(i+1))
    transformation = icp_p2l(point_cloud_arr[i], point_cloud_arr[i+1])
    transformation_arr.append(transformation)

### Composing Multiple Transformations for Reconstruction
Apply composed transformations on every point cloud to align them to the target

In [8]:
start_index = 0
target_index = len(point_cloud_arr) - 1
final_pc_arr = [None] * (target_index - start_index + 1)

for j in range(start_index, target_index):
#     print("Aligning point cloud " + str(j) + " to target point cloud " + str(target_index))
    pc_extracted = np.ones((4, cleaned_point_clouds[j].shape[0]))
    pc_extracted[0:3,:] = np.copy(cleaned_point_clouds[j].T)
    final_pc_arr[j] = pc_extracted

    composed_transformation = np.eye(4,4)
    for i in range(j, target_index):
        composed_transformation = np.dot(transformation_arr[i], composed_transformation)
        
    final_pc_arr[j] = np.dot(composed_transformation, final_pc_arr[j])[0:3,:]

pc_extracted_target = np.ones((4, cleaned_point_clouds[target_index].shape[0]))
pc_extracted_target[0:3,:] = np.copy(cleaned_point_clouds[target_index].T)

final_pc_arr[target_index] = pc_extracted_target[0:3,:]
final_reconstruction = np.hstack(final_pc_arr).T

## 4. Downsample and Visualize
Convert final point cloud to `py3d.PointCloud` objects in order to downsample and visualize

In [9]:
pcd = PointCloud()
pcd.points = Vector3dVector(final_reconstruction)
print("Downsample the point cloud with a voxel of 0.35")
downpcd = voxel_down_sample(pcd, voxel_size = 0.35)
downpcd

Downsample the point cloud with a voxel of 0.35


PointCloud with 1270716 points.

### Visualize Point Cloud

In [11]:
draw_geometries([downpcd])

### Save Point Cloud to .ply
For exporting or viewing in other programs like Meshlab

In [ ]:
# SAVE ORIGINAL POINT CLOUD

dest_ply_file_name = "final.ply"

with open(path + dest_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(final_reconstruction.shape[0]))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("end_header\n")
    
    for i in range(final_reconstruction.shape[0]):
        f.write("{} {} {}\n".format(final_reconstruction[i][0], 
                                    final_reconstruction[i][1], 
                                    final_reconstruction[i][2]))

pcd = read_point_cloud(path + "final.ply")

In [10]:
# SAVE DOWNSAMPLED POINT CLOUD

final_downsampled = np.asarray(downpcd.points)

dest_ply_file_name = "final-downsampled.ply"

with open(path + dest_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(final_downsampled.shape[0]))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("end_header\n")
    
    for i in range(final_downsampled.shape[0]):
        f.write("{} {} {}\n".format(final_downsampled[i][0], 
                                    final_downsampled[i][1], 
                                    final_downsampled[i][2]))